# Basic Usage

## Install evaltext

In [ ]:
pip install evaltext

# Run Code

In [ ]:
import logging

from dotenv import load_dotenv

from evaltext import GeneratorType
from evaltext import S
from evaltext.runner import create_runner

logging.basicConfig(level=logging.INFO)

load_dotenv()

runner = create_runner([
    GeneratorType.OPENAI_GPT_3_5_TURBO,
    GeneratorType.OPENAI_DAVINCI_003,
    GeneratorType.OPENAI_DAVINCI_002,
], verbose=True)


def scoring(text) -> float:
    return (
        S(text)
        .line(0, lambda s: s.not_contains_regex(r"(\w+):"))
        .contains("Action:")
        .contains("Action Input:")
        .not_contains("Action:", greater_than=1)
        .not_contains("Action Input:", greater_than=1)
        .not_contains("Final Answer:")
        .part(r"Action:\s*(.+)\s*", lambda s: s.one_of(['google_search', 'webpage_get', 'calculator']))
        .part_p(r"ActionInput:\s*(.+)\s*", lambda t: t.startswith("BOCCHI"))
        .score()
    )


# output results to 'output/score.jsonl' by default.
runner.measure('examples/prompts/prompt001.txt', scoring)